In [1]:
import pandas as pd
import numpy as np
import json, os, sys, glob, re
from tqdm import tqdm

### Create new datasets with ranked features

In [2]:
data = pd.read_csv('../data/X_train_499.csv')
print(data.shape)

(25546, 500)


In [3]:
# Load the rankings.
with open('decoded_ranks.json', 'r') as f:
    rankings = json.load(f)

# feature names = keys and ranks = values.
rank_dict = {v: int(k) for k, v in rankings.items()}

df = data

# (excluding 'ID').
columns = df.columns.tolist()[1:]

# The columns are sorted by ranking.
sorted_columns = sorted(columns, key=lambda x: rank_dict.get(x, float('inf')))
reordered_df = df[['ID'] + sorted_columns]
reordered_df.to_csv('../data/X_train_id_ranked.csv', index=False)
print("saved as 'data/X_train_id_ranked.csv'")

saved as 'data/X_train_id_ranked.csv'


In [ ]:
df = pd.read_csv('../data/X_train_id_ranked.csv')

# (excluding 'ID')
columns = df.columns.tolist()[1:]  

# Make directory.
os.makedirs('../data/train-subsets', exist_ok=True)

# Create subsets. 
for i in tqdm(range(len(columns), 0, -1), desc="Creating subsets"):
    subset_columns = ['ID'] + columns[:i]
    subset_df = df[subset_columns]
    filename = f'../data/train-subsets/X_train_id_top_{i}_features.csv'
    subset_df.to_csv(filename, index=False)

print("Saved in 'data/train-subsets'.")

In [4]:
test = pd.read_csv('../data/X_test_id.csv', low_memory=False)
print(test.shape)

(2831, 500)


In [5]:
# Load the rankings.
with open('decoded_ranks.json', 'r') as f:
    rankings = json.load(f)

# feature names as keys and ranks as values.
rank_dict = {v: int(k) for k, v in rankings.items()}

df = test
columns = df.columns.tolist()[1:] 
sorted_columns = sorted(columns, key=lambda x: rank_dict.get(x, float('inf')))
reordered_df = df[['ID'] + sorted_columns]
reordered_df.to_csv('../data/X_test_id_ranked.csv', index=False)

print("saved as 'data/X_test_id_ranked.csv'")

saved as 'data/X_test_id_ranked.csv'


In [8]:
df = pd.read_csv('../data/X_test_id_ranked.csv', low_memory=False)

# (excluding 'ID')
columns = df.columns.tolist()[1:]  

# Make directory.
os.makedirs('../data/test-subsets', exist_ok=True)

# Create subsets.
for i in tqdm(range(len(columns), 0, -1), desc="Creating test subsets"):
    subset_columns = ['ID'] + columns[:i]
    subset_df = df[subset_columns]
    filename = f'../data/test-subsets/X_test_id_top_{i}_features.csv'
    subset_df.to_csv(filename, index=False)

print("Saved in the 'data/subsets' directory.")

Creating test subsets: 100%|██████████| 499/499 [01:45<00:00,  4.74it/s]

All test subsets saved in the 'data/subsets' directory.


### For XGB, must encode feature names prior to use.

In [4]:
path = '../data/encoded/train'

In [17]:
# Find all train sets.
file_pattern = '../data/train-subsets/X_train_id_top_*_features.csv'
filenames = glob.glob(file_pattern)

def feature_count(filename):
    """
    Extracts feature count from the names.
    """
    match = re.search(r'top_(\d+)_features', filename)
    return int(match.group(1)) if match else 0

# Sort files from names.
filenames = sorted(filenames, key=feature_count)

# Init dict to store the names.
short_names = {}

# Use tqdm to populate the dict with progress bar.
for filename in tqdm(filenames, desc="Processing filenames"):
    base_name = os.path.basename(filename)
    if base_name.startswith('X_train_id_top_'):
        feature_count = feature_count(filename)
        short_name = f'X_train_{feature_count}'
        short_names[short_name] = filename

# Show shapes of the df's.
for short_name, filename in tqdm(short_names.items(), desc="Loading DataFrames"):
    globals()[short_name] = pd.read_csv(filename, low_memory=False)
   
# Show names.
df_names = list(short_names.keys())

# Add to list.
df_list = [globals()[name] for name in df_names]
print(f'\nTotal number of DataFrames: {len(df_list)}')
for i, df in enumerate(tqdm(df_list, desc="Confirming DataFrames")):
    print(f'Shape of {df_names[i]}: {df.shape}')

Loading DataFrames: 100%|██████████| 499/499 [02:45<00:00,  3.01it/s]



Total number of DataFrames: 499


Confirming DataFrames: 100%|██████████| 499/499 [00:00<00:00, 188368.08it/s]

Shape of X_train_1: (25546, 2)
Shape of X_train_2: (25546, 3)
Shape of X_train_3: (25546, 4)
Shape of X_train_4: (25546, 5)
Shape of X_train_5: (25546, 6)
Shape of X_train_6: (25546, 7)
Shape of X_train_7: (25546, 8)
Shape of X_train_8: (25546, 9)
Shape of X_train_9: (25546, 10)
Shape of X_train_10: (25546, 11)
Shape of X_train_11: (25546, 12)
Shape of X_train_12: (25546, 13)
Shape of X_train_13: (25546, 14)
Shape of X_train_14: (25546, 15)
Shape of X_train_15: (25546, 16)
Shape of X_train_16: (25546, 17)
Shape of X_train_17: (25546, 18)
Shape of X_train_18: (25546, 19)
Shape of X_train_19: (25546, 20)
Shape of X_train_20: (25546, 21)
Shape of X_train_21: (25546, 22)
Shape of X_train_22: (25546, 23)
Shape of X_train_23: (25546, 24)
Shape of X_train_24: (25546, 25)
Shape of X_train_25: (25546, 26)
Shape of X_train_26: (25546, 27)
Shape of X_train_27: (25546, 28)
Shape of X_train_28: (25546, 29)
Shape of X_train_29: (25546, 30)
Shape of X_train_30: (25546, 31)
Shape of X_train_31: (25546

In [5]:
X_train = pd.read_csv('../data/train-subsets/X_train_id_top_499_features.csv', low_memory=False)

In [19]:
def encode_names(df, original_names):
    """
    Use a dictionary to map numbers to names.
    """
    name_dict = {f"feature{i}": name for i, name in enumerate(original_names)}
    reverse_dict = {v: k for k, v in name_dict.items()}
    df_encoded = df.rename(columns=reverse_dict)
    return df_encoded, name_dict

# Retrieve the feature names.
original_names = globals()['X_train'].columns.tolist()

# Init directory for encoded files.
os.makedirs(path, exist_ok=True)

# Use tqdm to process with progress bar.
for df_name in tqdm(df_names, desc="Encoding and exporting DataFrames"):
    df = globals()[df_name]
    encoded_df, feature_dict = encode_names(df, original_names)
    output_file = f'{path}/{df_name}_encoded.csv'
    encoded_df.to_csv(output_file, index=False)
    print(f"Encoded DataFrame {df_name} has been exported to '{output_file}'")

# Export the mapping.
with open(f'{path}/feature_mapping.json', 'w') as f:
    json.dump(feature_dict, f, indent=2)
print(f"Feature name mapping has been saved to '{path}/feature_mapping.json'")

Encoding and exporting DataFrames:   1%|          | 4/499 [00:00<00:15, 31.44it/s]

Encoded DataFrame X_train_1 has been exported to '../data/encoded/train/X_train_1_encoded.csv'
Encoded DataFrame X_train_2 has been exported to '../data/encoded/train/X_train_2_encoded.csv'
Encoded DataFrame X_train_3 has been exported to '../data/encoded/train/X_train_3_encoded.csv'
Encoded DataFrame X_train_4 has been exported to '../data/encoded/train/X_train_4_encoded.csv'
Encoded DataFrame X_train_5 has been exported to '../data/encoded/train/X_train_5_encoded.csv'


Encoding and exporting DataFrames:   2%|▏         | 8/499 [00:00<00:22, 21.40it/s]

Encoded DataFrame X_train_6 has been exported to '../data/encoded/train/X_train_6_encoded.csv'
Encoded DataFrame X_train_7 has been exported to '../data/encoded/train/X_train_7_encoded.csv'
Encoded DataFrame X_train_8 has been exported to '../data/encoded/train/X_train_8_encoded.csv'
Encoded DataFrame X_train_9 has been exported to '../data/encoded/train/X_train_9_encoded.csv'


Encoding and exporting DataFrames:   2%|▏         | 11/499 [00:00<00:28, 17.17it/s]

...

Encoded DataFrame X_train_497 has been exported to '../data/encoded/train/X_train_497_encoded.csv'


Encoding and exporting DataFrames: 100%|█████████▉| 498/499 [15:30<00:03,  3.63s/it]

Encoded DataFrame X_train_498 has been exported to '../data/encoded/train/X_train_498_encoded.csv'


Encoding and exporting DataFrames: 100%|██████████| 499/499 [15:34<00:00,  1.87s/it]

Encoded DataFrame X_train_499 has been exported to '../data/encoded/train/X_train_499_encoded.csv'
Feature name mapping has been saved to '../data/encoded/train/feature_mapping.json'


In [6]:
NEGONE_FEATURES = ['MOD_RES','REGION','INTERACTION_REGION','REQUIRED_FOR_INTER','ATP_binding_gbind','Ca2+_binding_gbind','DNA_binding_gbind','HEME_binding_gbind','Mg2+_binding_gbind','Mn2+_binding_gbind','RNA_binding_gbind','Dist2Mutation','BLOSUM62','ProteinLengthChange','TSSDistance','1000Gp3_AF','UK10K_AF','gnomAD_exomes_AF','gnomAD_genomes_AF','MSC_95CI','rel_cDNA_pos','rel_CDS_pos','rel_prot_pos','GDI','Selective_pressure','Clarks_distance','CDS_len','Number_of_paralogs','denovo_Zscore','RVIS','Indispensability_score','RSA','ASA','RSA_Zfit','before_RSA_3','before_RSA_8','before_RSA_15','after_RSA_3','after_RSA_8','after_RSA_15','before_ASA_3','before_ASA_8','before_ASA_15','after_ASA_3','after_ASA_8','after_ASA_15','Phosphorylation','Acetylation','Methylation','Ubiquitination','Glycosylation','PTM','AF_Relative_ASA','IUPRED2','ANCHOR2','before_IUPRED_3','before_IUPRED_8','before_IUPRED_15','after_IUPRED_3','after_IUPRED_8','after_IUPRED_15','before_ANCHOR_3','before_ANCHOR_8','before_ANCHOR_15','after_ANCHOR_3','after_ANCHOR_8','after_ANCHOR_15','A3D_SCORE','n_contacts','distance_com','concavity_score','S_DDG[SEQ]','S_DDG[3D]','hgmd_mutcount','gnomsingle_mutcount','gnom_mutcount','AF_confidence','isHomomultimer','num_interactions','ppi_combined_0','ppi_combined_1','ppi_combined_2','ppi_combined_3','ppi_combined_4','ppi_combined_5','ppi_combined_6','ppi_combined_7','ppi_combined_8','ppi_combined_9','ppi_combined_10','ppi_combined_11','ppi_combined_12','ppi_combined_13','ppi_combined_14','ppi_combined_15','ppi_combined_16','ppi_combined_17','ppi_combined_18','ppi_combined_19','ppi_combined_20','ppi_combined_21','ppi_combined_22','ppi_combined_23','ppi_combined_24','ppi_combined_25','ppi_combined_26','ppi_combined_27','ppi_combined_28','ppi_combined_29','ppi_combined_30','ppi_combined_31','ppi_combined_32','ppi_combined_33','ppi_combined_34','ppi_combined_35','ppi_combined_36','ppi_combined_37','ppi_combined_38','ppi_combined_39','ppi_combined_40','ppi_combined_41','ppi_combined_42','ppi_combined_43','ppi_combined_44','ppi_combined_45','ppi_combined_46','ppi_combined_47','ppi_combined_48','ppi_combined_49','ppi_combined_50','ppi_combined_51','ppi_combined_52','ppi_combined_53','ppi_combined_54','ppi_combined_55','ppi_combined_56','ppi_combined_57','ppi_combined_58','ppi_combined_59','ppi_combined_60','ppi_combined_61','ppi_combined_62','ppi_combined_63','s_het','DRNApredDNAscore_aa_window_3_prev','DRNApredDNAscore_aa_window_8_prev','DRNApredDNAscore_aa_window_15_prev','DRNApredDNAscore_aa_window_3_next','DRNApredDNAscore_aa_window_8_next','DRNApredDNAscore_aa_window_15_next','DRNApredDNAscore_aa','ASAquick_normscore_aa_window_3_prev','ASAquick_normscore_aa_window_8_prev','ASAquick_normscore_aa_window_15_prev','ASAquick_normscore_aa_window_3_next','ASAquick_normscore_aa_window_8_next','ASAquick_normscore_aa_window_15_next','ASAquick_normscore_aa','ASAquick_rawscore_aa_window_3_prev','ASAquick_rawscore_aa_window_8_prev','ASAquick_rawscore_aa_window_15_prev','ASAquick_rawscore_aa_window_3_next','ASAquick_rawscore_aa_window_8_next','ASAquick_rawscore_aa_window_15_next','ASAquick_rawscore_aa','DFLpredScore_aa_window_3_prev','DFLpredScore_aa_window_8_prev','DFLpredScore_aa_window_15_prev','DFLpredScore_aa_window_3_next','DFLpredScore_aa_window_8_next','DFLpredScore_aa_window_15_next','DFLpredScore_aa','DRNApredRNAscore_aa_window_3_prev','DRNApredRNAscore_aa_window_8_prev','DRNApredRNAscore_aa_window_15_prev','DRNApredRNAscore_aa_window_3_next','DRNApredRNAscore_aa_window_8_next','DRNApredRNAscore_aa_window_15_next','DRNApredRNAscore_aa','DisoDNAscore_aa_window_3_prev','DisoDNAscore_aa_window_8_prev','DisoDNAscore_aa_window_15_prev','DisoDNAscore_aa_window_3_next','DisoDNAscore_aa_window_8_next','DisoDNAscore_aa_window_15_next','DisoDNAscore_aa','DisoPROscore_aa_window_3_prev','DisoPROscore_aa_window_8_prev','DisoPROscore_aa_window_15_prev','DisoPROscore_aa_window_3_next','DisoPROscore_aa_window_8_next','DisoPROscore_aa_window_15_next','DisoPROscore_aa','DisoRNAscore_aa_window_3_prev','DisoRNAscore_aa_window_8_prev','DisoRNAscore_aa_window_15_prev','DisoRNAscore_aa_window_3_next','DisoRNAscore_aa_window_8_next','DisoRNAscore_aa_window_15_next','DisoRNAscore_aa','MMseq2_conservation_level_aa_window_3_prev','MMseq2_conservation_level_aa_window_8_prev','MMseq2_conservation_level_aa_window_15_prev','MMseq2_conservation_level_aa_window_3_next','MMseq2_conservation_level_aa_window_8_next','MMseq2_conservation_level_aa_window_15_next','MMseq2_conservation_level_aa','MMseq2_conservation_score_aa_window_3_prev','MMseq2_conservation_score_aa_window_8_prev','MMseq2_conservation_score_aa_window_15_prev','MMseq2_conservation_score_aa_window_3_next','MMseq2_conservation_score_aa_window_8_next','MMseq2_conservation_score_aa_window_15_next','MMseq2_conservation_score_aa','MoRFchibiScore_aa_window_3_prev','MoRFchibiScore_aa_window_8_prev','MoRFchibiScore_aa_window_15_prev','MoRFchibiScore_aa_window_3_next','MoRFchibiScore_aa_window_8_next','MoRFchibiScore_aa_window_15_next','MoRFchibiScore_aa','PSIPRED_helix_aa_window_3_prev','PSIPRED_helix_aa_window_8_prev','PSIPRED_helix_aa_window_15_prev','PSIPRED_helix_aa_window_3_next','PSIPRED_helix_aa_window_8_next','PSIPRED_helix_aa_window_15_next','PSIPRED_helix_aa','PSIPRED_strand_aa_window_3_prev','PSIPRED_strand_aa_window_8_prev','PSIPRED_strand_aa_window_15_prev','PSIPRED_strand_aa_window_3_next','PSIPRED_strand_aa_window_8_next','PSIPRED_strand_aa_window_15_next','PSIPRED_strand_aa','SCRIBERscore_aa_window_3_prev','SCRIBERscore_aa_window_8_prev','SCRIBERscore_aa_window_15_prev','SCRIBERscore_aa_window_3_next','SCRIBERscore_aa_window_8_next','SCRIBERscore_aa_window_15_next','SCRIBERscore_aa','SignalP_score_aa_window_3_prev','SignalP_score_aa_window_8_prev','SignalP_score_aa_window_15_prev','SignalP_score_aa_window_3_next','SignalP_score_aa_window_8_next','SignalP_score_aa_window_15_next','SignalP_score_aa','gtex_Adipose_-_Subcutaneous','gtex_Adipose_-_Visceral_(Omentum)','gtex_Adrenal_Gland','gtex_Artery_-_Aorta','gtex_Artery_-_Coronary','gtex_Artery_-_Tibial','gtex_Bladder','gtex_Brain_-_Amygdala','gtex_Brain_-_Anterior_cingulate_cortex_(BA24)','gtex_Brain_-_Caudate_(basal_ganglia)','gtex_Brain_-_Cerebellar_Hemisphere','gtex_Brain_-_Cerebellum','gtex_Brain_-_Cortex','gtex_Brain_-_Frontal_Cortex_(BA9)','gtex_Brain_-_Hippocampus','gtex_Brain_-_Hypothalamus','gtex_Brain_-_Nucleus_accumbens_(basal_ganglia)','gtex_Brain_-_Putamen_(basal_ganglia)','gtex_Brain_-_Spinal_cord_(cervical_c-1)','gtex_Brain_-_Substantia_nigra','gtex_Breast_-_Mammary_Tissue','gtex_Cells_-_Cultured_fibroblasts','gtex_Cells_-_EBV-transformed_lymphocytes','gtex_Cervix_-_Ectocervix','gtex_Cervix_-_Endocervix','gtex_Colon_-_Sigmoid','gtex_Colon_-_Transverse','gtex_Esophagus_-_Gastroesophageal_Junction','gtex_Esophagus_-_Mucosa','gtex_Esophagus_-_Muscularis','gtex_Fallopian_Tube','gtex_Heart_-_Atrial_Appendage','gtex_Heart_-_Left_Ventricle','gtex_Kidney_-_Cortex','gtex_Kidney_-_Medulla','gtex_Liver','gtex_Lung','gtex_Minor_Salivary_Gland','gtex_Muscle_-_Skeletal','gtex_Nerve_-_Tibial','gtex_Ovary','gtex_Pancreas','gtex_Pituitary','gtex_Prostate','gtex_Skin_-_Not_Sun_Exposed_(Suprapubic)','gtex_Skin_-_Sun_Exposed_(Lower_leg)','gtex_Small_Intestine_-_Terminal_Ileum','gtex_Spleen','gtex_Stomach','gtex_Testis','gtex_Thyroid','gtex_Uterus','gtex_Vagina','gtex_Whole_Blood','haplo','haplo_imputed','PHOSPHORYLATION','ACETYLATION','UBIQUITINATION','S-NITROSYLATION','N-GLYCOSYLATION','METHYLATION','O-GLYCOSYLATION','MYRISTOYLATION','C-GLYCOSYLATION','SUMOYLATION','S-GLYCOSYLATION','polyphen_nobs','polyphen_normasa','polyphen_dvol','polyphen_dprop','polyphen_bfact','polyphen_hbonds','polyphen_avenhet','polyphen_mindhet','polyphen_avenint','polyphen_mindint','polyphen_avensit','polyphen_mindsit','polyphen_idpmax','polyphen_idpsnp','polyphen_idqmin','motifECount','motifEHIPos','motifEScoreChng','Dst2Splice','motifDist','EncodeH3K4me1-sum','EncodeH3K4me1-max','EncodeH3K4me2-sum','EncodeH3K4me2-max','EncodeH3K4me3-sum','EncodeH3K4me3-max','EncodeH3K9ac-sum','EncodeH3K9ac-max','EncodeH3K9me3-sum','EncodeH3K9me3-max','EncodeH3K27ac-sum','EncodeH3K27ac-max','EncodeH3K27me3-sum','EncodeH3K27me3-max','EncodeH3K36me3-sum','EncodeH3K36me3-max','EncodeH3K79me2-sum','EncodeH3K79me2-max','EncodeH4K20me1-sum','EncodeH4K20me1-max','EncodeH2AFZ-sum','EncodeH2AFZ-max','EncodeDNase-sum','EncodeDNase-max','EncodetotalRNA-sum','EncodetotalRNA-max','Grantham_x','Freq100bp','Rare100bp','Sngl100bp','Freq1000bp','Rare1000bp','Sngl1000bp','Freq10000bp','Rare10000bp','Sngl10000bp','RemapOverlapTF','RemapOverlapCL','Charge','Volume','Hydrophobicity','Polarity','Ex','PAM250','JM','HGMD2003','VB','Transition','COSMIC','COSMICvsSWISSPROT','HAPMAP','COSMICvsHAPMAP',]
MEDIAN_FEATURES = ['CADD_raw','Conservation','MaxEntScan_alt','MaxEntScan_diff','MaxEntScan_ref','ada_score','rf_score','FATHMM_score','GERPplus_plus_NR','GERPplus_plus_RS','GM12878_fitCons_score','GenoCanyon_score','H1_hESC_fitCons_score','HUVEC_fitCons_score','LINSIGHT','LIST_S2_score','LRT_score','M_CAP_score','MPC_score','MVP_score','MutationAssessor_score','MutationTaster_score','PROVEAN_score','SiPhy_29way_logOdds','VEST4_score','fathmm_MKL_coding_score','fathmm_XF_coding_score','integrated_fitCons_score','phastCons100way_vertebrate','phastCons17way_primate','phastCons30way_mammalian','phyloP100way_vertebrate','phyloP17way_primate','phyloP30way_mammalian','Condel_score','SIFT_score','NearestExonJB_distance','NearestExonJB_len','Dominant_probability','Recessive_probability','polyphen_dscore','polyphen_score1','polyphen_score2','ConsScore','GC','CpG','minDistTSS','minDistTSE','priPhCons','mamPhCons','verPhCons','priPhyloP','mamPhyloP','verPhyloP','bStatistic_y','targetScan','mirSVR-Score','mirSVR-E','mirSVR-Aln','cHmm_E1','cHmm_E2','cHmm_E3','cHmm_E4','cHmm_E5','cHmm_E6','cHmm_E7','cHmm_E8','cHmm_E9','cHmm_E10','cHmm_E11','cHmm_E12','cHmm_E13','cHmm_E14','cHmm_E15','cHmm_E16','cHmm_E17','cHmm_E18','cHmm_E19','cHmm_E20','cHmm_E21','cHmm_E22','cHmm_E23','cHmm_E24','cHmm_E25','GerpRS','GerpRSpval','GerpN','GerpS','tOverlapMotifs','SpliceAI-acc-gain','SpliceAI-acc-loss','SpliceAI-don-gain','SpliceAI-don-loss','MMSp_acceptorIntron','MMSp_acceptor','MMSp_exon','MMSp_donor','MMSp_donorIntron','dbscSNV-ada_score','dbscSNV-rf_score',]

In [21]:
def feature_lists(feature_lists, feature_dict):
    """ 
    Encode the NEGONE and MEDIAN feature names.
    """
    reverse_dict = {v: k for k, v in feature_dict.items()}
    encoded_lists = []
    for feature_list in feature_lists:
        encoded_list = []
        for feature in feature_list:
            if feature in reverse_dict:
                encoded_list.append(reverse_dict[feature])
            else:
                print(f"Warning: Feature '{feature}' not found in the mapping.")
        encoded_lists.append(encoded_list)
    return encoded_lists

# Do the encoding.
encoded_negone, encoded_median = feature_lists([NEGONE_FEATURES, MEDIAN_FEATURES], feature_dict)

# Place encoded lists inside a dictionary.
encoded_features = {
    "NEGONE_FEATURES": encoded_negone,
    "MEDIAN_FEATURES": encoded_median
}

# Export to JSON.
output_path = '../data/negone_median.json'
with open(output_path, 'w') as json_file:
    json.dump(encoded_features, json_file, indent=4)
print(f"Encoded features have been saved to '{output_path}'")

Encoded features have been saved to '../data/negone_median.json'


In [ ]:
""" Now, encode the other features mentioned in the LoGoFunc pre-processing script. """

patterns = ['after', 'before', 'gnomAD_exomes_AF', 'gnomAD_genomes_AF']

# Search for features that match the patterns.
matching_features = {}
for key, value in feature_dict.items():
    if any(pattern in value for pattern in patterns):
        matching_features[key] = value

output_path = '../data/patterns.json'

# Make directory.
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Export to JSON.
with open(output_path, 'w') as json_file:
    json.dump(matching_features, json_file, indent=4)
print(f"Saved to '{output_path}'")

In [23]:
""" Repeat process for ID and IMPACT columns, because they are mentioned in the LoGoFunc pre-processing script. """
""" These will be manually added to the pre-processing pipeline. """

patterns = ['ID', 'IMPACT']

# Search for features that match the patterns.
matching_keys = []
for key, value in feature_dict.items():
    if any(pattern in value for pattern in patterns):
        matching_keys.append(key)

print("Matching keys:")
for key in matching_keys:
    print(f"{key}: {feature_dict[key]}")

Matching keys:
feature0: ID
feature2: IMPACT


In [24]:
# Find all test sets.
file_pattern = '../data/test-subsets/X_test_id_top_*_features.csv'
filenames = glob.glob(file_pattern)

def feature_count(filename):
    """ 
    Extracts feature count from the names.
    """
    match = re.search(r'top_(\d+)_features', filename)
    return int(match.group(1)) if match else 0

# Sort files from names.
filenames = sorted(filenames, key=feature_count)

# Init dict to store the names.
short_names = {}

# Use tqdm to populate the dict with progress bar.
for filename in tqdm(filenames, desc="Processing test filenames"):
    base_name = os.path.basename(filename)
    if base_name.startswith('X_test_id_top_'):
        feature_count = feature_count(filename)
        short_name = f'X_test_{feature_count}'
        short_names[short_name] = filename

# Show shapes of the df's.
for short_name, filename in tqdm(short_names.items(), desc="Loading test DataFrames"):
    globals()[short_name] = pd.read_csv(filename, low_memory=False)
    print(f'Shape of {short_name}: {globals()[short_name].shape}')

# Show names.
df_names = list(short_names.keys())

# Add to list.
df_list = [globals()[name] for name in df_names]
print(f'\nTotal number of DataFrames: {len(df_list)}')
for i, df in enumerate(tqdm(df_list, desc="Confirming test DataFrames")):
    print(f'Shape of {df_names[i]}: {df.shape}')

Loading test DataFrames:   6%|▌         | 28/499 [00:00<00:01, 279.13it/s]

Shape of X_test_1: (2831, 2)
Shape of X_test_2: (2831, 3)
Shape of X_test_3: (2831, 4)
Shape of X_test_4: (2831, 5)
Shape of X_test_5: (2831, 6)
Shape of X_test_6: (2831, 7)
Shape of X_test_7: (2831, 8)
Shape of X_test_8: (2831, 9)
Shape of X_test_9: (2831, 10)
Shape of X_test_10: (2831, 11)
Shape of X_test_11: (2831, 12)
Shape of X_test_12: (2831, 13)
Shape of X_test_13: (2831, 14)
Shape of X_test_14: (2831, 15)
Shape of X_test_15: (2831, 16)
Shape of X_test_16: (2831, 17)
Shape of X_test_17: (2831, 18)
Shape of X_test_18: (2831, 19)
Shape of X_test_19: (2831, 20)
Shape of X_test_20: (2831, 21)
Shape of X_test_21: (2831, 22)
Shape of X_test_22: (2831, 23)
Shape of X_test_23: (2831, 24)
Shape of X_test_24: (2831, 25)
Shape of X_test_25: (2831, 26)
Shape of X_test_26: (2831, 27)
Shape of X_test_27: (2831, 28)
Shape of X_test_28: (2831, 29)
Shape of X_test_29: (2831, 30)
Shape of X_test_30: (2831, 31)
Shape of X_test_31: (2831, 32)
Shape of X_test_32: (2831, 33)
Shape of X_test_33: (2831

Loading test DataFrames:  11%|█         | 56/499 [00:00<00:02, 179.54it/s]

Shape of X_test_45: (2831, 46)
Shape of X_test_46: (2831, 47)
Shape of X_test_47: (2831, 48)
Shape of X_test_48: (2831, 49)
Shape of X_test_49: (2831, 50)
Shape of X_test_50: (2831, 51)
Shape of X_test_51: (2831, 52)
Shape of X_test_52: (2831, 53)
Shape of X_test_53: (2831, 54)
Shape of X_test_54: (2831, 55)
Shape of X_test_55: (2831, 56)
Shape of X_test_56: (2831, 57)
Shape of X_test_57: (2831, 58)
Shape of X_test_58: (2831, 59)
Shape of X_test_59: (2831, 60)
Shape of X_test_60: (2831, 61)
Shape of X_test_61: (2831, 62)
Shape of X_test_62: (2831, 63)
Shape of X_test_63: (2831, 64)
Shape of X_test_64: (2831, 65)
Shape of X_test_65: (2831, 66)
Shape of X_test_66: (2831, 67)


Loading test DataFrames:  15%|█▌        | 76/499 [00:00<00:03, 132.79it/s]

...

Shape of X_test_495: (2831, 496)
Shape of X_test_496: (2831, 497)
Shape of X_test_497: (2831, 498)


Loading test DataFrames: 100%|██████████| 499/499 [00:18<00:00, 26.41it/s]


Shape of X_test_498: (2831, 499)
Shape of X_test_499: (2831, 500)

Total number of DataFrames: 499


Confirming test DataFrames: 100%|██████████| 499/499 [00:00<00:00, 238187.97it/s]

Shape of X_test_1: (2831, 2)
Shape of X_test_2: (2831, 3)
Shape of X_test_3: (2831, 4)
Shape of X_test_4: (2831, 5)
Shape of X_test_5: (2831, 6)
Shape of X_test_6: (2831, 7)
Shape of X_test_7: (2831, 8)
Shape of X_test_8: (2831, 9)
Shape of X_test_9: (2831, 10)
Shape of X_test_10: (2831, 11)
Shape of X_test_11: (2831, 12)
Shape of X_test_12: (2831, 13)
Shape of X_test_13: (2831, 14)
Shape of X_test_14: (2831, 15)
Shape of X_test_15: (2831, 16)
Shape of X_test_16: (2831, 17)
Shape of X_test_17: (2831, 18)
Shape of X_test_18: (2831, 19)
Shape of X_test_19: (2831, 20)
Shape of X_test_20: (2831, 21)
Shape of X_test_21: (2831, 22)
Shape of X_test_22: (2831, 23)
Shape of X_test_23: (2831, 24)
Shape of X_test_24: (2831, 25)
Shape of X_test_25: (2831, 26)
Shape of X_test_26: (2831, 27)
Shape of X_test_27: (2831, 28)
Shape of X_test_28: (2831, 29)
Shape of X_test_29: (2831, 30)
Shape of X_test_30: (2831, 31)
Shape of X_test_31: (2831, 32)
Shape of X_test_32: (2831, 33)
Shape of X_test_33: (2831

In [25]:
X_test = pd.read_csv('../data/test-subsets/X_test_id_top_499_features.csv', low_memory=False)
path = '../data/encoded/test'

In [26]:
def encode_names(df, original_names):
    """
    Use a dictionary to map numbers to names.
    """
    name_dict = {f"feature{i}": name for i, name in enumerate(original_names)}
    reverse_dict = {v: k for k, v in name_dict.items()}
    df_encoded = df.rename(columns=reverse_dict)
    return df_encoded, name_dict

# Retrieve the feature names.
original_names = globals()['X_test'].columns.tolist()

# Init directory for encoded files.
os.makedirs(path, exist_ok=True)

# Use tqdm to process with progress bar.
for df_name in tqdm(df_names, desc="Encoding and exporting test DataFrames"):
    df = globals()[df_name]
    encoded_df, feature_dict = encode_names(df, original_names)
    output_file = f'{path}/{df_name}_encoded.csv'
    encoded_df.to_csv(output_file, index=False)
    print(f"Encoded DataFrame {df_name} has been exported to '{output_file}'")

# Export the mapping.
with open(f'{path}/feature_mapping.json', 'w') as f:
    json.dump(feature_dict, f, indent=2)
print(f"Feature name mapping has been saved to '{path}/feature_mapping.json'")

Encoding and exporting test DataFrames:   3%|▎         | 13/499 [00:00<00:03, 122.28it/s]

Encoded DataFrame X_test_1 has been exported to '../data/encoded/test/X_test_1_encoded.csv'
Encoded DataFrame X_test_2 has been exported to '../data/encoded/test/X_test_2_encoded.csv'
Encoded DataFrame X_test_3 has been exported to '../data/encoded/test/X_test_3_encoded.csv'
Encoded DataFrame X_test_4 has been exported to '../data/encoded/test/X_test_4_encoded.csv'
Encoded DataFrame X_test_5 has been exported to '../data/encoded/test/X_test_5_encoded.csv'
Encoded DataFrame X_test_6 has been exported to '../data/encoded/test/X_test_6_encoded.csv'
Encoded DataFrame X_test_7 has been exported to '../data/encoded/test/X_test_7_encoded.csv'
Encoded DataFrame X_test_8 has been exported to '../data/encoded/test/X_test_8_encoded.csv'
Encoded DataFrame X_test_9 has been exported to '../data/encoded/test/X_test_9_encoded.csv'
Encoded DataFrame X_test_10 has been exported to '../data/encoded/test/X_test_10_encoded.csv'
Encoded DataFrame X_test_11 has been exported to '../data/encoded/test/X_test_

Encoding and exporting test DataFrames:   5%|▌         | 26/499 [00:00<00:06, 74.17it/s] 

Encoded DataFrame X_test_20 has been exported to '../data/encoded/test/X_test_20_encoded.csv'
Encoded DataFrame X_test_21 has been exported to '../data/encoded/test/X_test_21_encoded.csv'
Encoded DataFrame X_test_22 has been exported to '../data/encoded/test/X_test_22_encoded.csv'
Encoded DataFrame X_test_23 has been exported to '../data/encoded/test/X_test_23_encoded.csv'
Encoded DataFrame X_test_24 has been exported to '../data/encoded/test/X_test_24_encoded.csv'
Encoded DataFrame X_test_25 has been exported to '../data/encoded/test/X_test_25_encoded.csv'
Encoded DataFrame X_test_26 has been exported to '../data/encoded/test/X_test_26_encoded.csv'
Encoded DataFrame X_test_27 has been exported to '../data/encoded/test/X_test_27_encoded.csv'
Encoded DataFrame X_test_28 has been exported to '../data/encoded/test/X_test_28_encoded.csv'
Encoded DataFrame X_test_29 has been exported to '../data/encoded/test/X_test_29_encoded.csv'


Encoding and exporting test DataFrames:   7%|▋         | 35/499 [00:00<00:08, 56.79it/s]

...

Encoded DataFrame X_test_497 has been exported to '../data/encoded/test/X_test_497_encoded.csv'


Encoding and exporting test DataFrames: 100%|█████████▉| 498/499 [01:45<00:00,  2.50it/s]

Encoded DataFrame X_test_498 has been exported to '../data/encoded/test/X_test_498_encoded.csv'


Encoding and exporting test DataFrames: 100%|██████████| 499/499 [01:45<00:00,  4.73it/s]

Encoded DataFrame X_test_499 has been exported to '../data/encoded/test/X_test_499_encoded.csv'
Feature name mapping has been saved to '../data/encoded/test/feature_mapping.json'
